# Upload DIVERSE Model Logs to HuggingFace

DIVERSE model training loglarını HuggingFace'e yükler

In [ ]:
# Kurulum
!pip install -q huggingface_hub

from google.colab import drive
from huggingface_hub import HfApi, create_repo
import os

drive.mount('/content/drive')
print("✓ Drive bağlandı")

In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()
print("✓ HuggingFace'e giriş yapıldı")

In [ ]:
# DIVERSE Model için repo hazırla
api = HfApi()

try:
    create_repo(
        repo_id="bodhai/qwen-coder-lora-diverse",
        repo_type="model",
        exist_ok=True
    )
    print("✓ DIVERSE repo hazır")
except Exception as e:
    print(f"✓ DIVERSE repo zaten var: {e}")

In [ ]:
# DIVERSE logs yükle
diverse_log_dir = '/content/drive/MyDrive/lora_checkpoints/diverse/logs'

print(f"Log klasörü kontrol ediliyor: {diverse_log_dir}")

if os.path.exists(diverse_log_dir):
    # Dosyaları listele
    files = os.listdir(diverse_log_dir)
    print(f"\nBulunan dosyalar: {files}")
    
    # Yükle
    print("\nYükleniyor...")
    api.upload_folder(
        folder_path=diverse_log_dir,
        repo_id="bodhai/qwen-coder-lora-diverse",
        repo_type="model",
        path_in_repo="logs"
    )
    print("\n✅ DIVERSE logs başarıyla yüklendi!")
    print("\n🔗 Kontrol et: https://huggingface.co/bodhai/qwen-coder-lora-diverse/tree/main/logs")
else:
    print(f"\n❌ Log klasörü bulunamadı: {diverse_log_dir}")
    print("\nDrive yapısını kontrol et:")
    !ls -la /content/drive/MyDrive/lora_checkpoints/